# LAB | Imbalanced

**Load the data**

In this challenge, we will be working with Credit Card Fraud dataset.

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv

Metadata

- **distance_from_home:** the distance from home where the transaction happened.
- **distance_from_last_transaction:** the distance from last transaction happened.
- **ratio_to_median_purchase_price:** Ratio of purchased price transaction to median purchase price.
- **repeat_retailer:** Is the transaction happened from same retailer.
- **used_chip:** Is the transaction through chip (credit card).
- **used_pin_number:** Is the transaction happened by using PIN number.
- **online_order:** Is the transaction an online order.
- **fraud:** Is the transaction fraudulent. **0=legit** -  **1=fraud**


In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [2]:
fraud = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv")
fraud.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


**Steps:**

- **1.** What is the distribution of our target variable? Can we say we're dealing with an imbalanced dataset?
- **2.** Train a LogisticRegression.
- **3.** Evaluate your model. Take in consideration class importance, and evaluate it by selection the correct metric.
- **4.** Run **Oversample** in order to balance our target variable and repeat the steps above, now with balanced data. Does it improve the performance of our model? 
- **5.** Now, run **Undersample** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model?
- **6.** Finally, run **SMOTE** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model? 

In [3]:
#1

# Miro la distribución de la variable 'fraud' para ver si está desbalanceado
# Con normalize=True veo los porcentajes, que es más claro
distribucion = fraud['fraud'].value_counts(normalize=True)
print("Distribución de la variable objetivo (fraud):")
print(distribucion)

# Conclusión: Sí, está muy desbalanceado. Más del 91% de los casos son 'no fraude' (0).

Distribución de la variable objetivo (fraud):
fraud
0.0    0.912597
1.0    0.087403
Name: proportion, dtype: float64


In [4]:
# Preparo los datos para los modelos
# X son todas las columnas menos la que quiero predecir
X = fraud.drop('fraud', axis=1)
# y es la columna que quiero predecir
y = fraud['fraud']

# Divido los datos en entrenamiento y prueba. 
# Importante: esta división se hace una sola vez sobre los datos originales.
# Las técnicas de balanceo solo se aplicarán a X_train y y_train.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
#2 y 3

# Entreno un modelo de Regresión Logística con los datos originales
modelo_original = LogisticRegression(max_iter=1000, random_state=42)
modelo_original.fit(X_train, y_train)

# Hago predicciones y evalúo
predicciones_original = modelo_original.predict(X_test)

print("Resultados del modelo con datos desbalanceados:")
print(classification_report(y_test, predicciones_original))

# Conclusión: La 'accuracy' es muy alta (96%), pero es una métrica engañosa aquí.
# Lo importante es el 'recall' para la clase 1 (fraude), que es solo 0.60.
# Esto significa que el modelo solo está detectando el 60% de los fraudes reales, se le escapan muchos.

Resultados del modelo con datos desbalanceados:
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98    273779
         1.0       0.90      0.60      0.72     26221

    accuracy                           0.96    300000
   macro avg       0.93      0.80      0.85    300000
weighted avg       0.96      0.96      0.96    300000



In [ ]:
#4

# Aplico Oversampling para balancear los datos de entrenamiento
oversampler = RandomOverSampler(random_state=42)
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

# Entreno un nuevo modelo con los datos balanceados por oversampling
modelo_over = LogisticRegression(max_iter=1000, random_state=42)
modelo_over.fit(X_train_over, y_train_over)

# Evalúo el modelo en el conjunto de test (que sigue desbalanceado, como en la realidad)
predicciones_over = modelo_over.predict(X_test)

print("Resultados del modelo con Oversampling:")
print(classification_report(y_test, predicciones_over))

# Conclusión: El 'recall' para la clase 1 ha mejorado de forma espectacular, pasando de 0.60 a 0.95.
# Ahora el modelo detecta el 95% de los fraudes. Sin embargo, la 'precision' ha bajado a 0.58,
# lo que significa que ahora hay más "falsas alarmas".

c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


Resultados del modelo con Oversampling:
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    273779
         1.0       0.58      0.95      0.72     26221

    accuracy                           0.93    300000
   macro avg       0.79      0.94      0.84    300000
weighted avg       0.96      0.93      0.94    300000



In [ ]:
#5

# Aplico Undersampling para balancear los datos de entrenamiento
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

# Entreno un nuevo modelo con los datos balanceados por undersampling
modelo_under = LogisticRegression(max_iter=1000, random_state=42)
modelo_under.fit(X_train_under, y_train_under)

# Evalúo el modelo
predicciones_under = modelo_under.predict(X_test)

print("Resultados del modelo con Undersampling:")
print(classification_report(y_test, predicciones_under))

# Conclusión: Los resultados son idénticos a los de Oversampling.
# El 'recall' para la clase 1 también sube a 0.95, mejorando muchísimo la detección de fraudes
# a cambio de una menor precisión (0.58).

c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


Resultados del modelo con Undersampling:
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    273779
         1.0       0.58      0.95      0.72     26221

    accuracy                           0.93    300000
   macro avg       0.79      0.94      0.84    300000
weighted avg       0.96      0.93      0.94    300000



In [ ]:
#6

# Aplico SMOTE, una técnica de oversampling más avanzada
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Entreno un nuevo modelo con los datos generados por SMOTE
modelo_smote = LogisticRegression(max_iter=1000, random_state=42)
modelo_smote.fit(X_train_smote, y_train_smote)

# Evalúo el modelo
predicciones_smote = modelo_smote.predict(X_test)

print("Resultados del modelo con SMOTE:")
print(classification_report(y_test, predicciones_smote))

# Conclusión: SMOTE también produce los mismos resultados que las otras dos técnicas.
# El 'recall' de 0.95 demuestra que es muy efectivo para encontrar fraudes.
#
# Veredicto final: Las tres técnicas (Oversampling, Undersampling y SMOTE) han mejorado
# drásticamente la capacidad del modelo para detectar fraudes, y en este caso,
# todas han llegado a un resultado idéntico. Cualquiera de ellas sería una buena elección.

c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Resultados del modelo con SMOTE:
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    273779
         1.0       0.58      0.95      0.72     26221

    accuracy                           0.93    300000
   macro avg       0.79      0.94      0.84    300000
weighted avg       0.96      0.93      0.94    300000

